Import Libraries and data

In [3]:
import importlib

import optuna
from scipy.sparse import csr_matrix
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from Optimize.SaveResults import SaveResults
from Optimize.slim import objective_function_SLIM
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
import pandas as pd
import scipy.sparse as sps
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens1MReader import Movielens1MReader
from Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_SVDpp_Cython
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
import Optimize
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

# Press Maiusc+F10 to execute it or replace it with your code.
# Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.


URM_all_dataframe = pd.read_csv(filepath_or_buffer="Data/data_train.csv",
                                sep=",",
                                dtype={0:int, 1:int, 2:float},
                                engine='python')
users = pd.read_csv(filepath_or_buffer="Data/data_target_users_test.csv")

ICM = pd.read_csv(filepath_or_buffer="Data/data_ICM_metadata.csv",
                                sep=",",
                                dtype={0:int, 1:int, 2:float},
                                engine='python')




n_interactions = len(URM_all_dataframe)
mapped_id, original_id = pd.factorize(URM_all_dataframe["user_id"].unique())
user_original_ID_to_index = pd.Series(mapped_id, index=original_id)
mapped_id, original_id = pd.factorize(URM_all_dataframe["item_id"].unique())
item_original_ID_to_index = pd.Series(mapped_id, index=original_id)
print(URM_all_dataframe['item_id'].max())
print(URM_all_dataframe.head(4))
#URM_all_dataframe["user_id"] = URM_all_dataframe["user_id"].map(user_original_ID_to_index)
#URM_all_dataframe["item_id"] = URM_all_dataframe["item_id"].map(item_original_ID_to_index)
print(URM_all_dataframe['item_id'].max())
print(URM_all_dataframe.head(4))
userID_unique = URM_all_dataframe["user_id"].unique()
itemID_unique = URM_all_dataframe["item_id"].unique()
n_users = len(userID_unique)
n_items = len(itemID_unique)
print(n_users, n_items)
URM_all = sps.coo_matrix((URM_all_dataframe["data"].values,
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)))
n_items = ICM["item_id"].max() + 1
n_features = ICM["feature_id"].max() + 1
ICM_all = sps.coo_matrix((ICM["data"].values, (ICM["item_id"].values, ICM["feature_id"].values)) , shape=(n_items, n_features))

URM_all.tocsr()
ICM_all.tocsr()

URM_trainval, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_trainval, train_percentage = 0.8)


38120
   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
38120
   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
35736 38121


In [4]:
slimrec = SLIMElasticNetRecommender(URM_trainval)
slimrec.fit(alpha= 0.0002021210695683939, topK= 856, l1_ratio= 0.23722934371355184)

SLIMElasticNetRecommender: Processed 1910 ( 5.0%) in 5.00 min. Items per second: 6.36
SLIMElasticNetRecommender: Processed 3876 (10.2%) in 10.00 min. Items per second: 6.46
SLIMElasticNetRecommender: Processed 5839 (15.3%) in 15.00 min. Items per second: 6.49
SLIMElasticNetRecommender: Processed 7814 (20.5%) in 20.00 min. Items per second: 6.51
SLIMElasticNetRecommender: Processed 9785 (25.7%) in 25.01 min. Items per second: 6.52
SLIMElasticNetRecommender: Processed 11734 (30.8%) in 30.01 min. Items per second: 6.52
SLIMElasticNetRecommender: Processed 13744 (36.1%) in 35.01 min. Items per second: 6.54
SLIMElasticNetRecommender: Processed 15763 (41.3%) in 40.01 min. Items per second: 6.57
SLIMElasticNetRecommender: Processed 17734 (46.5%) in 45.01 min. Items per second: 6.57
SLIMElasticNetRecommender: Processed 19740 (51.8%) in 50.01 min. Items per second: 6.58
SLIMElasticNetRecommender: Processed 21764 (57.1%) in 55.02 min. Items per second: 6.59
SLIMElasticNetRecommender: Processed 2

In [5]:
from Recommenders.ScoresHybridRecommender import ScoresHybridRecommender
evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

result_df, _ = evaluator_validation.evaluateRecommender(slimrec)

EvaluatorHoldout: Ignoring 144 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35592 (100.0%) in 36.23 sec. Users per second: 982


In [6]:
result_df

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.111241,0.166205,0.141262,0.060819,0.088535,0.311383,0.163484,0.124467,0.572151,0.42726,...,0.99597,0.569846,0.99597,0.23461,13.409933,0.999863,0.385652,0.908413,1.396552,0.243511


In [7]:
graphrec = RP3betaRecommender(URM_trainval)
graphrec.fit(topK= 12, alpha= 0.5769111396825488, beta= 0.0019321798490027353)

RP3betaRecommender: Similarity column 38121 (100.0%), 1926.07 column/sec. Elapsed time 19.79 sec


In [8]:
from Recommenders.ScoresHybridRecommender import ScoresHybridRecommender
evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

result_df, _ = evaluator_validation.evaluateRecommender(graphrec)

EvaluatorHoldout: Ignoring 144 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35592 (100.0%) in 28.28 sec. Users per second: 1259


In [9]:
result_df

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.100981,0.151249,0.128961,0.054017,0.078881,0.28777,0.148373,0.113268,0.540599,0.387601,...,0.99597,0.538421,0.99597,0.216638,13.306463,0.999852,0.356111,0.901404,1.339414,0.24397


In [10]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
itemKnncf = ItemKNNCFRecommender(URM_trainval)
itemKnncf.fit(similarity= "cosine", topK= 8, shrink= 12)

Cython module imported successfully.
Similarity column 38121 (100.0%), 1939.46 column/sec. Elapsed time 19.66 sec


In [11]:
evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

result_df, _ = evaluator_validation.evaluateRecommender(graphrec)
result_df

EvaluatorHoldout: Ignoring 144 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35592 (100.0%) in 29.02 sec. Users per second: 1226


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.100981,0.151249,0.128961,0.054017,0.078881,0.28777,0.148373,0.113268,0.540599,0.387601,...,0.99597,0.538421,0.99597,0.216638,13.306463,0.999852,0.356111,0.901404,1.339414,0.24397


In [12]:
type(graphrec.W_sparse)

scipy.sparse.csr.csr_matrix

In [13]:
type(slimrec.W_sparse)

scipy.sparse.csr.csr_matrix

In [14]:
from Recommenders.HybridOptunable2 import HybridOptunable2


def  obj_hybrid(optuna_trial):
    print("helloworld")
    alpha = optuna_trial.suggest_float("alpha", 0.1, 0.9)
    recommender_object = HybridOptunable2(URM_trainval)
    recommender_object.fit(alpha,itemKnncf, slimrec)
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_object)
    return result_df.loc[10][ "MAP"]


In [15]:
study = optuna.create_study(direction="maximize")
save_results = SaveResults()
study.optimize(obj_hybrid, callbacks= [save_results], n_trials=50)

[I 2024-11-29 03:55:23,222] A new study created in memory with name: no-name-3d05aac2-1414-4a99-9a91-15326f98c42f


helloworld
EvaluatorHoldout: Processed 35592 (100.0%) in 36.84 sec. Users per second: 966


[I 2024-11-29 03:56:00,227] Trial 0 finished with value: 0.06056660672810184 and parameters: {'alpha': 0.8081014129940781}. Best is trial 0 with value: 0.06056660672810184.


helloworld
EvaluatorHoldout: Processed 35592 (100.0%) in 36.01 sec. Users per second: 988


[I 2024-11-29 03:56:36,377] Trial 1 finished with value: 0.05981677820412446 and parameters: {'alpha': 0.7300433839234144}. Best is trial 0 with value: 0.06056660672810184.


...

[I 2024-11-29 04:24:13,363] Trial 47 finished with value: 0.05984033330122143 and parameters: {'alpha': 0.7332448857105416}. Best is trial 32 with value: 0.061131682659201124.


helloworld
EvaluatorHoldout: Processed 35592 (100.0%) in 35.53 sec. Users per second: 1002


[I 2024-11-29 04:24:49,026] Trial 48 finished with value: 0.05546160720975097 and parameters: {'alpha': 0.40042355679166186}. Best is trial 32 with value: 0.061131682659201124.


helloworld
EvaluatorHoldout: Processed 35592 (100.0%) in 36.15 sec. Users per second: 985


[I 2024-11-29 04:25:25,334] Trial 49 finished with value: 0.060945090434089265 and parameters: {'alpha': 0.8682874726062778}. Best is trial 32 with value: 0.061131682659201124.


In [19]:
from ModelNames import ModelName
from Recommenders.HybridOptunable2 import HybridOptunable2
recommender_object = HybridOptunable2(URM_all)
recommender_object.fit(0.8989445006775345, itemKnncf ,slimrec)
result_df, _ = evaluator_validation.evaluateRecommender(recommender_object)

recommender_object.save_model(folder_path="_saved_models", file_name = ModelName.HybridOptunable2)



EvaluatorHoldout: Processed 35592 (100.0%) in 24.46 sec. Users per second: 1455


In [22]:
result_df

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.111505,0.166456,0.141328,0.061132,0.088936,0.311935,0.163867,0.124658,0.571055,0.428553,...,0.99597,0.568754,0.99597,0.242922,13.471256,0.999877,0.399315,0.912567,1.329641,0.244657


In [23]:
recommendations_list = []
cutoff = 10
for user_id in users["user_id"]:
    recommendations = recommender_object.recommend(user_id, cutoff=cutoff)
    recommendations_list.append([user_id, recommendations])



result_df, _ = evaluator_validation.evaluateRecommender(recommender_object)
print (result_df.loc[10]["MAP"])
df_recommendations = pd.DataFrame(recommendations_list, columns=['user_id', 'item_list'])
df_recommendations.to_csv('recomm.csv', index=False)

EvaluatorHoldout: Processed 35592 (100.0%) in 25.01 sec. Users per second: 1423
0.061131682659201124
